In [1]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch
import time

In [2]:
torch.cuda.empty_cache()

In [3]:
model_name = "google/pegasus-xsum"
model = PegasusForConditionalGeneration.from_pretrained(model_name).to('cuda')
tokenizer = PegasusTokenizer.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jaysa\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
og_med_data = pd.read_json("hf://datasets/ncbi/Open-Patients/Open-Patients.jsonl", lines=True)
og_med_data.head()

,_id,description
0,trec-cds-2014-1,A 58-year-old African-American woman presents ...
1,trec-cds-2014-2,An 8-year-old male presents in March to the ER...
2,trec-cds-2014-3,A 58-year-old nonsmoker white female with mild...
3,trec-cds-2014-4,A 2-year-old boy is brought to the emergency d...
4,trec-cds-2014-5,A 56-year-old female on 20th day post-left mas...


In [5]:
og_med_data_subset = og_med_data.iloc[0:30000,:]

In [6]:
def generate_pegasus_summary_batch(texts, batch_size=4, max_input_length=1024, max_output_length=150):
    summaries = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, truncation=True, padding=True, max_length=max_input_length, return_tensors="pt").to("cuda")
        outputs = model.generate(inputs["input_ids"], max_length=max_output_length, num_beams=4, early_stopping=True)
        decoded_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        summaries.extend(decoded_batch)
    return summaries

In [7]:
def tokenize_texts(texts):
    return [" ".join(nltk.word_tokenize(text.lower())) for text in texts]

In [8]:
descriptions = og_med_data_subset["description"].tolist()

In [9]:
def calculate_sentence_bleu(reference, generated):
    smoothing_function = SmoothingFunction().method1
    scores = [
        sentence_bleu([nltk.word_tokenize(ref)], nltk.word_tokenize(gen), smoothing_function=smoothing_function)
        for ref, gen in zip(reference, generated)
    ]
    return sum(scores) / len(scores)

In [10]:
start_time = time.time()
pegasus_summaries = generate_pegasus_summary_batch(descriptions, batch_size=2)
pegasus_time = time.time() - start_time

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
reference_tokenized = tokenize_texts(descriptions)
generated_tokenized = tokenize_texts(pegasus_summaries)

In [12]:
sentence_bleu_score = calculate_sentence_bleu(reference_tokenized, generated_tokenized)

In [13]:
print(f"PEGASUS Summary Generation Metrics:")
print(f"  Time taken: {pegasus_time:.2f} seconds")
print(f"  Sentence-Level BLEU Score: {sentence_bleu_score:.4f}")

PEGASUS Summary Generation Metrics:
  Time taken: 293.11 seconds
  Sentence-Level BLEU Score: 0.0011


In [21]:
avg_rouge1, avg_rouge2, avg_rougeL = og_med_data_subset[['rouge1','rouge2','rougeL']].mean()

In [22]:
print(f"Average Test Rouge1 Score: {avg_rouge1:.4f}")
print(f"Average Test Rouge2 Score: {avg_rouge2:.4f}")
print(f"Average Test RougeL Score: {avg_rougeL:.4f}")

Average Test Rouge1 Score: 0.1189
Average Test Rouge2 Score: 0.0530
Average Test RougeL Score: 0.0902


In [16]:
for ref, gen in zip(reference_tokenized[:5], generated_tokenized[:5]):
    print("\n---")
    print(f"Reference: {ref}")
    print(f"Generated: {gen}")


---
Reference: a 58-year-old african-american woman presents to the er with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life . the pain started while she was walking , radiates to the back , and is accompanied by nausea , diaphoresis and mild dyspnea , but is not increased on inspiration . the latest episode of pain ended half an hour prior to her arrival . she is known to have hypertension and obesity . she denies smoking , diabetes , hypercholesterolemia , or a family history of heart disease . she currently takes no medications . physical examination is normal . the ekg shows nonspecific changes .
Generated: here is a case report of a patient who presents with chest pain for the first time in her life .

---
Reference: an 8-year-old male presents in march to the er with fever up to 39 c , dyspnea and cough for 2 days . he has just returned from a 5 day vacation in colorado . parents report that prior to the onset of fever and 

In [13]:
og_med_data_subset['pegasus_summary'] = pegasus_summaries
og_med_data_subset.to_csv('pegasus_generated_summaries.csv', index=False)

C:\Users\jaysa\AppData\Local\Temp\ipykernel_17092\2405544121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  og_med_data_subset['pegasus_summary'] = pegasus_summaries


In [ ]:
print("Generated summaries saved to 'pegasus_generated_summaries.csv'")